In [1]:
struct Delta
    n::Int
    q::Int
    avg::Vector{Vector{Float64}}
end;

In [2]:
# initializes correct size delta
function Delta(n::Int, q::Int)
    Delta(n, q, [zeros(Float64, q * q) for _ in 1:(n * n)])
end;

In [3]:
# cartesian to linear indices conversion
function cart2lin(t::Tuple{Int, Int}, dims::Int)
    x, y = t[2], t[1]
    @assert 1 ≤ x ≤ dims && 1 ≤ y ≤ dims
    return y + dims * (x-1)
end;

In [4]:
cart2lin(i::Int, j::Int, dims::Int) = cart2lin((i, j),dims);

In [5]:
# updates the delta structure when the condition is verified
function Delta_update!(δ::Delta, i::Int, j::Int, a::Int, b::Int)
    n = δ.n
    q = δ.q
    δ.avg[cart2lin((i,j), n)][cart2lin((a,b), q)] += 1
    nothing
end;

In [6]:
function compute_stat(data::Matrix{Int64}, q::Int)
	count = size(data, 1)
	n = size(data, 2)
	delta_ijab = Delta(n, q)

	for i in 1:n, j in 1:n, a in 1:q, b in 1:q
		[Delta_update!(delta_ijab, i, j, a, b) for s in 1:count if (data[s, i] == a) && (data[s, j] == b)]
	end
	delta_ijab.avg /= count

	return delta_ijab
end;

In [7]:
function maxabs_vecvec(δ::Delta)
	m = δ.avg
	max = -1
	for r in m
		if maximum(abs.(m)) > max
			max = maximum(abs.(m))
		end
	end
	return max
end;

In [8]:
function metropolis_hastings_step(x::Vector{Int64}, J_struct::Delta)
	n = length(x)
    J = J_struct.avg
	n = J_stuct.n
	q = J_Struct.q
    
	# 1. draw uniformly an index and a new configuration
	k = rand(1:n)
	z = rand(1:q)
	
	# 2. compute the acceptance ratio
	a = 0
	xk_new = mod1(x[k] + z, q)
	for i in 1:n
		if i != k
			a += J[cart2lin(i, k, n)][cart2lin(x[i], xk_new, q)] - J[cart2lin(i, k, n)][cart2lin(x[i], x[k], q)]
		end
	end
	a *= 2
	a += J[cart2lin(k, k, n)][cart2lin(xk_new, xk_new, q)] - J[cart2lin(k, k, n)][cart2lin(x[k], x[k], q)]
	a = exp(a)
	a = min(1, a)

	# 3/4. decide whether to accept or not the new configuration
	if rand() < a
		x[k] = xk_new
	end
	
	return x
end;